In [1]:
import pandas as pd
import math
import numpy as np
from python_tsp.exact import solve_tsp_brute_force
# https://github.com/fillipe-gsm/python-tsp
# https://networkx.org/documentation/stable/index.html
import folium
from folium.plugins import MarkerCluster
from IPython.display import HTML

In [2]:
def map_generator(df,zoom_start=10):
    start_point = [df.iloc[0].lat,df.iloc[0].lon]
    map = folium.Map(location=start_point, zoom_start = zoom_start)
    marker_cluster = MarkerCluster().add_to(map)
    for idx, row in df.iterrows():
        folium.CircleMarker(
            location=[row['lat'], 
            row['lon']], 
            radius = 9, 
            popup=row['info'], 
            fill_color=row['fill_color'], 
            color=row['color'], 
            fill_opacity = row['fill_opacity']
        ).add_to(marker_cluster)
    return map, map._repr_html_()

#### In:  
1. last master coordinate
2. list of vacant bids

#### Out:
1. number of next bid

### Data preparing

In [3]:
data = [
    {
        'info':'Courier 0',
        'lat':55.8197038,
        'lon':37.5544834
    },
    {
        'info':'1762984',
        'lat':55.765189,
        'lon':37.563781
    },
    {
        'info':'1763000',
        'lat':55.844465,
        'lon':37.492742
    },
    {
        'info':'1762994',
        'lat':55.839689,
        'lon':37.399578
    },
    {
        'info':'1763006',
        'lat':55.862338,
        'lon':37.608940
    },
    {
        'info':'1762997',
        'lat':55.773839,
        'lon':37.588907
    },
    {
        'info':'1763013',
        'lat':55.678128,
        'lon':37.770735
    }    
]
df = pd.DataFrame(data)
df['fill_color'] = np.ones(len(df))*255
df['color'] = np.ones(len(df))*0
df['fill_opacity'] = np.ones(len(df))*0.7
#df.loc[df['info']=='Courier 0','color'] = 255
#df.loc[df['info']=='Courier 0','fill_color'] = 0
df

,info,lat,lon,fill_color,color,fill_opacity
0,Courier 0,55.819704,37.554483,255.0,0.0,0.7
1,1762984,55.765189,37.563781,255.0,0.0,0.7
2,1763000,55.844465,37.492742,255.0,0.0,0.7
3,1762994,55.839689,37.399578,255.0,0.0,0.7
4,1763006,55.862338,37.608940,255.0,0.0,0.7
5,1762997,55.773839,37.588907,255.0,0.0,0.7
6,1763013,55.678128,37.770735,255.0,0.0,0.7


In [4]:
m, generated_map = map_generator(df)
HTML(generated_map)

### Convert points to distance graph

In [5]:
def distance(coordinate1, coordinate2):
    """
    :param coordinate1: gps coordinate
    :param coordinate2: gps coordinate
    :return: distance between two gps coordinates
    """
    return math.sqrt((coordinate1[0] - coordinate2[0]) ** 2 + (coordinate1[1] - coordinate2[1]) ** 2)

In [6]:
coordinates = []
for idx, row in df.iterrows():
    coordinates.append([row.lat,row.lon])

In [7]:
coordinates

[[55.8197038, 37.5544834],
 [55.765189, 37.563781],
 [55.844465, 37.492742],
 [55.839689, 37.399578],
 [55.862338, 37.60894],
 [55.773839, 37.588907],
 [55.678128, 37.770735]]

In [8]:
adjacency_matrix = []
for i in range(len(coordinates)):
    adjacency_matrix.append([])
    for j in range(len(coordinates)):
        adjacency_matrix[i].append(distance(coordinates[i], coordinates[j]))

In [9]:
adjacency_matrix = np.array(adjacency_matrix)

In [10]:
permutation, distance = solve_tsp_brute_force(adjacency_matrix)
permutation, distance

([0, 2, 3, 1, 5, 6, 4], 0.8865103905885666)

### Plot the route

In [11]:
points = []
for i in permutation:
    p = permutation[i]    
    points.append((coordinates[p][0], coordinates[p][1]))
points

[(55.8197038, 37.5544834),
 (55.839689, 37.399578),
 (55.765189, 37.563781),
 (55.844465, 37.492742),
 (55.678128, 37.770735),
 (55.862338, 37.60894),
 (55.773839, 37.588907)]

In [12]:
start_point = [df.iloc[0].lat,df.iloc[0].lon]
#m = folium.Map(location=start_point, zoom_start = 10)
# https://python-visualization.github.io/folium/modules.html#folium.vector_layers.PolyLine
folium.PolyLine(points).add_to(m)
route_map = m._repr_html_()
m